전이학습 - Fine Tuning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader

In [2]:
# Set Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('device info >> ', device)

device info >>  cpu


In [3]:
# Load pre-trained model
model = models.resnet50(pretrained=True)
# print(model)

"""
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=2048, out_features=1000, bias=True)
)
"""

# Freeze model parameters -> 매개변수 고정
for param in model.parameters() :
    param.requires_grad = False # 역전파 과정 중에서 기울기 계산이 필요하지 여부 깃발

# Modify the last layer for the number of classes in your new task
num_classes = 10
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=num_classes)

# print(model)

"""
(2): Bottleneck(
      (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=2048, out_features=10, bias=True)
)
"""

/Users/park.s.w/anaconda3/envs/NN2/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/park.s.w/anaconda3/envs/NN2/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


'\n(2): Bottleneck(\n      (conv1): Conv2d(2048, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)\n      (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n      (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)\n      (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n      (conv3): Conv2d(512, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)\n      (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n      (relu): ReLU(inplace=True)\n    )\n  )\n  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))\n  (fc): Linear(in_features=2048, out_features=10, bias=True)\n)\n'

In [4]:
# Load and preprocess the data
transform = transforms.Compose([
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize((0.5,0.5,0.5,),(0.2,0.2,0.2,))
])

train_dataset = torchvision.datasets.CIFAR10(root='../99_StudyFiles/02_Data/', train=True, download=False, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='../99_StudyFiles/02_Data/', train=False, download=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [5]:
# loss fuction and optimiser
criterion  = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.fc.parameters(), lr=0.001)

model = model.to(device)
# print(model)

In [6]:
# training loop
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = 0.0
    model.train() # model을 train mode로 전환
    
    for images, labels in train_loader :
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        
    train_loss /= len(train_loader.dataset)
    print(f'Epoch : {epoch+1}/{num_epochs}, Train Loss : {train_loss}')

Epoch : 1/5, Train Loss : 0.7562683358764648


KeyboardInterrupt: 

In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader :
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (pred == labels).sum().item()
        
acc = 100 * correct / total
print(f'Test acc : {acc:.2f}%')